In [1]:
import nltk

# Download the CoNLL 2003 NER dataset (if not downloaded)
nltk.download('conll2003')

from nltk.corpus import conll2003

# Load the CoNLL 2003 NER dataset
sentences = conll2003.sents()
ne_chunks = conll2003.iob_sents()

# Accessing sentences and named entity chunks
for i in range(5):  # Print the first 5 sentences and named entity chunks
    print(f"Sentence {i + 1}: {sentences[i]}")
    print(f"Named Entities {i + 1}: {ne_chunks[i]}")
    print("\n" + "="*50 + "\n")


[nltk_data] Error loading conll2003: <urlopen error [WinError 10060] A
[nltk_data]     connection attempt failed because the connected party
[nltk_data]     did not properly respond after a period of time, or
[nltk_data]     established connection failed because connected host
[nltk_data]     has failed to respond>


ImportError: cannot import name 'conll2003' from 'nltk.corpus' (c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\corpus\__init__.py)

In [1]:
!pip install sklearn_crfsuite

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python37\python.exe -m pip install --upgrade pip' command.


In [2]:
import pandas as pd

In [3]:
df = pd.read_json("./ta_test.json", lines=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 758 entries, 0 to 757
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   words   758 non-null    object
 1   ner     758 non-null    object
dtypes: object(2)
memory usage: 12.0+ KB


In [5]:
df.isna().value_counts()

words  ner  
False  False    758
dtype: int64

In [6]:
dataset = []
for _, row in df.iterrows():
    dataset.append(list(zip(row["words"], row["ner"])))

In [8]:
dataset[1]

[('மாதவரம்', 'B-LOC'), (',', 'O'), ('திருவொற்றியூர்', 'B-LOC')]

In [10]:
import sklearn_crfsuite
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import re


# Split data into training and testing sets
train_data, test_data = train_test_split(dataset, test_size=0.2, random_state=42)

def extract_features(sentence, index):
    word, ner_tag = sentence[index]
    
    return {
        'word': word,
        'is_first': index == 0,
        'is_last': index == len(sentence) - 1,
        'prefix-1': "" if not word else word[0],
        'prefix-2': word[:2],
        'prefix-3': word[:3],
        'prefix-4': word[:4],
        'suffix-1': "" if not word else word[-1],
        'suffix-2': word[-2:],
        'suffix-3': word[-3:],
        'suffix-4': word[-4:],
        'prev_word': '' if index == 0 else sentence[index - 1][0],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1][0],
        'has_hyphen': '-' in word,
        'is_numeric': word.isdigit(),

    }

def sent2features(sent):
    tmp = []
    for i in range(len(sent)):
        tmp.append(extract_features(sent, i))
    return tmp

def sent2labels(sent):
    return [label for _, label in sent]

# Extract features and labels from the training and testing sets
X_train = [sent2features(sent) for sent in train_data]
y_train = [sent2labels(sent) for sent in train_data]
X_test = [sent2features(sent) for sent in test_data]
y_test = [sent2labels(sent) for sent in test_data]


In [12]:

# Initialize the CRF model
crf = sklearn_crfsuite.CRF(algorithm='lbfgs',
                           c1=0.1,
                           c2=0.1,
                           max_iterations=100,
                           all_possible_transitions=True)

# Train the CRF model
try:
    crf.fit(X_train, y_train)
except AttributeError:
    print("ok")
    pass


In [21]:
import pickle
with open('ner_crf_model.pkl', 'wb') as file:
    pickle.dump(crf, file)

In [13]:

# Make predictions on the test set
y_pred = crf.predict(X_test)

# Evaluate the model



In [20]:
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
print("## Penn ##")

#First calculate a prediction from test data, then we print the metrics for f-1 using the .flat_f1_score method.
y_pred=crf.predict(X_test)
print("F1 score on Test Data")
print(metrics.flat_f1_score(y_test, y_pred,average='weighted',labels=crf.classes_))
#For the sake of clarification, we do the same for train data.
y_pred_train=crf.predict(X_train)
print("F1 score on Training Data ")
print(metrics.flat_f1_score(y_train, y_pred_train,average='weighted',labels=crf.classes_))

## Penn ##
F1 score on Test Data
0.8425170088639261


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\metrics\_classification.py:1757: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


F1 score on Training Data 
0.998345072631972
